# Markov Chain

**Learning Objectives:** Learn how to build Markov Chains from n-grams and generate new sequences from the Markov Chains.

In [1]:
import types
from itertools import islice
import random

## n-grams

You can read about the background information related to n-grams [here](https://en.wikipedia.org/wiki/N-gram). Write a function, `build_ngrams`, that returns n-grams from an input sequene (iterator). Try to do this without concrete lists. The `islice` function may be helpful.

In [2]:
def build_ngrams(itr, n=2):
    """Return the sequence of n-grams from the source iterator."""
    for ndx, i in enumerate(itr):
        if ndx <= len(itr) - n:
            yield(tuple(islice(itr, ndx, ndx+n)))

In [3]:
a = build_ngrams(range(10), n=2)
assert hasattr(a, '__iter__') and not isinstance(a, list)
al = list(a)
assert al == [(i,i+1) for i in range(9)]

b = build_ngrams(range(10), n=5)
assert hasattr(b, '__iter__') and not isinstance(b, list)
bl = list(b)
assert bl == [(i,i+1,i+2,i+3,i+4) for i in range(6)]

assert list(build_ngrams('one two three four five six seven'.split(' '), n=5)) == \
    [('one','two','three','four','five'),
     ('two','three','four','five','six'),
     ('three','four','five','six','seven')]

## Markov chain

You can read about the background of Markov Chains [here](https://en.wikipedia.org/wiki/Markov_chain). Write a function `build_chain`, that returns a Markov Chain for a sequence of n-grams. This function should return a `dict` with:

* The keys will be the source node in the Markov Chain, which is the first `n-1` elements in each n-gram, as a `tuple`.
* The values will be a *list* of possible targets in the Markov Chain. As you find new values for a single key, you will need to append to the list.

In [4]:
def build_chain(ngrams, chain=None):
    """Build a Markov chain out of an iterator of n-grams.
    
    Parameters
    ----------
    ngrams: list of n-tuples
        A list of n-grams as tuples, where the first n-1 elements are the source node
        in the Markov chain ahd the last element is the target node in the Markov chain.
    chain: dict or None
        An existing Markov chain to add ngrams to or None for a new chain.
    """
    if not chain:
        chain = {}
    for gram in ngrams:
        current = gram[0:-1]
        last = gram[-1]
        if current not in chain:
            chain[current] = [last]
        else:
            chain[current].append(last)
    return chain

In [5]:
random.seed(0)
seq1 = [random.randint(0,10) for i in range(200)]
chain = build_chain(build_ngrams(seq1, n=3))
seq2 = [random.randint(0,10) for i in range(200)]
chain = build_chain(build_ngrams(seq2, n=3), chain=chain)
assert chain[(0,0)]==[7, 10, 0, 3, 4]
assert chain[(4,2)]==[1, 3, 8, 3, 7, 1, 10, 2, 8]
assert len(chain.keys())==111

Write a function, `generate_sequence`, that can generate new sequences of length `m` from a trained Markov Chain (in the `dict` format described above). For the initial part of the sequence, randomly choose one of the keys in the Markov Chain `dict`.

In [6]:
import random

def generate_sequence(chain, m):
    """Generate a new sequence of length n from a Markov chain.
    
    Parameter
    ---------
    chain : dict
        A dict where the keys are the source node of the Markov chain steps and
        the values are a list of possible targets.
    m : int
        The length of the sequence to generate.
    """
    start = random.choice(list(chain.keys()))
    for w in start:
        yield w
    for i in range(m - len(start)):
        n = random.choice(chain[start])
        yield n
        start = (start[1:]) + (n,)

In [7]:
random.seed(0)
seq3 = [random.randint(0,10) for i in range(200)]
chain2 = build_chain(build_ngrams(seq1, n=3))
assert list(generate_sequence(chain2, 10))==[8, 0, 1, 8, 10, 6, 8, 4, 8, 9]
chain3 = build_chain(build_ngrams(seq1, n=5))
assert list(generate_sequence(chain3, 10))==[4, 1, 8, 5, 8, 3, 9, 8, 9, 4]

## Scrape the web to find lyrics

For this part of the exercise, you will need to find lyrics from one of your favorite bands, and use the [requests](http://docs.python-requests.org/en/latest/) and [BeautifulSoup](http://www.crummy.com/software/BeautifulSoup/bs4/doc/) packages to scrape the lyrics from a website. Some guidance:

1. The more songs the better (many dozens).
2. No hand downloading or editing of the files, you must do everything from code.
3. Save all of the lyrics in a single text file in this directory.

I provide an example here of doing this for all of U2's lyrics. You will have to modify this code significantly for the band of your choice.

In [8]:
import requests
from bs4 import BeautifulSoup

First get the page that has an index of all the lyrics and create a list of the URLs of those pages:

In [9]:
def get_lyric_urls():
    index = requests.get("http://www.lyricsfreak.com/e/erasure/")
    soup = BeautifulSoup(index.text, 'html.parser')
    lyric_paths = [link.get('href') for link in soup.find_all('table')[1].find_all('a')]
    lyric_urls = ['http://www.lyricsfreak.com'+path for path in lyric_paths]
    return lyric_urls

In [10]:
lyric_urls = get_lyric_urls()[3:]

In [11]:
lyric_urls.remove('http://www.lyricsfreak.com/e/erasure/the+soldiers+return_20050868.html')
lyric_urls.remove('http://www.lyricsfreak.com/e/erasure/runaround+on+the+underground_20050862.html')

Here is a function that takes the URL of a single lyric page and scrapes the actual lyric as text:

In [12]:
def get_lyric(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    return str(soup.find_all(id="content_h")[0])

This gets all of the lyrics. Note the `time.sleep(1.0)`. When scraping websites, it is often important to throttle your requests so as to not get banned from the website.

In [13]:
import time

def get_all_lyrics(lyric_urls):
    for url in lyric_urls:
        time.sleep(1.0)
        yield get_lyric(url)

In [14]:
lyrics = list(get_all_lyrics(lyric_urls))

KeyboardInterrupt: 

In [ ]:
clean = [BeautifulSoup(str(lyric).replace('<br>', '\n'), "html.parser") for lyric in lyrics]    

Now save all the lyrics to a text file:

In [ ]:
with open('all_erasure_lyrics.txt', 'w') as f:
    for l in clean:
        e = l.getText()
        f.write(e)
        f.write('\n')

Leave the following cell to grade your code for this section:

In [ ]:
assert True

## Generate new lyrics with the Markov chain

Here is the fun part!

In [15]:
import textwrap

Here are some simple function for tokenizing the lyrics:

In [16]:
from quicktoken import tokenize_lines, tokenize_line

Read in your lyric file, tokenize the text (no stop words!) and generate new song lyrics. Some things to think about:

* This will work best if you generate new lines of text of some finite length (10-30 words).
* Use `textwrap.wrap` to format these lines and separate them using newlines.
* To get interesting results, you may need to run it multiple times.

In [17]:
def file_to_lines(file):
    """Iterator over a sequence of filenames, yielding all of the lines in the files."""
    with open(file, 'r') as text:
        for l in text:
            yield(l)

In [18]:
c = build_chain(build_ngrams(list(tokenize_lines(file_to_lines('all_erasure_lyrics.txt'))), n=3))

In [21]:
for i in range(5):
    print(textwrap.wrap(' '.join(generate_sequence(c,30)), 30))

["ooh sometimes it's the last", 'dance searching for your heart', 'oh that i want to break that', "heart of mine for keeps i'm", 'trying to hold you back here', 'in']
['it tonight chorus wave goodbye', "lost for you mon amour what's", 'a boy in the air can you feel', 'joy can you really mean it and', 'feel the tear drops']
['lot we depend on a brave face', 'but it means to me i will try', 'to go down to you you hold', "your head what's on your knees", 'and then']
['the cannon fire lost voices', 'echo in the eyes of blue what', "you say don't go beating me", 'like you do with the joys of', 'spring no love is too']
['over a new morality make some', 'improvement on the shore then', 'rolls out to me angel talk', 'with me play with me time with', 'me and my love go with']
